In [2]:
# 필요한 라이브러리 호출
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt

# 데이터 불러오기
df = pd.read_csv("C:/msys64/home/for/10th/standard/LinearBNB/2025_Airbnb_NYC_listings.csv")

In [17]:
# 전처리 
# 가격 '$', ',' 때고 숫자형으로 바꾸기
df['price'] = df['price'].str.replace('$', '').str.replace(',', '')
df['price'] = df['price'].astype(float)
df['price'] = df['price'].astype(int)

# 'bathrooms', 'bedrooms', 'beds', 'review_scores_rating' 중 하나라도 결측치가 있는 '행(row)'을 제거
df.dropna(subset=['bathrooms', 'bedrooms', 'beds', 'review_scores_rating'], inplace=True)

# 너무 오래된 리뷰 데이터는 제거
old_date = pd.to_datetime('2023-03-02')
df['last_review'] = pd.to_datetime(df['last_review'])
df = df[df['last_review'] >= old_date].copy()


df.reset_index(drop=True, inplace=True)


C:\Users\for\AppData\Local\Temp\ipykernel_22924\1400678716.py:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['price'] = df['price'].str.replace('$', '').str.replace(',', '')


In [4]:
features =['neighbourhood_cleansed',       # 정제된 동네 이름
            'neighbourhood_group_cleansed', # 자치구 단위 지역 그룹 (맨해튼, 브루클린 등)
            'property_type',                # 숙소 유형 (아파트, 집 등)
            'room_type',                    # 방 유형 (전체, 개인실 등)
            'accommodates',                 # 수용 인원
            'bathrooms',                    # 욕실 개수             
            'bedrooms',                     # 침실 개수
            'beds',                         # 침대 개수
            'amenities',                    # 편의 시설
            'minimum_nights',               # 최소 숙박 일수
            'maximum_nights',
            'price']              # 최대 숙박 일수

In [5]:
df[features].isna().sum()

neighbourhood_cleansed          0
neighbourhood_group_cleansed    0
property_type                   0
room_type                       0
accommodates                    0
bathrooms                       0
bedrooms                        0
beds                            0
amenities                       0
minimum_nights                  0
maximum_nights                  0
price                           0
dtype: int64

In [6]:
df = df[features]

In [7]:
df.shape

(13531, 12)

# 파생 컬럼 생성

In [8]:
# 지역적 특성을 통해 지역을 그룹화 하기

geo_mapping = {
    # 1. 중심 관광/비즈니스 (High-Density, Global Hub)
    'Midtown': 'Global_Hub', 'Hell\'s Kitchen': 'Global_Hub', 'Chelsea': 'Global_Hub', 
    'Financial District': 'Global_Hub', 'Theater District': 'Global_Hub', 'Murray Hill': 'Global_Hub',
    'Kips Bay': 'Global_Hub', 'West Village': 'Global_Hub', 'SoHo': 'Global_Hub', 'Chinatown': 'Global_Hub',

    # 2. 힙스터/예술/트렌디 (Trendy & Cultural)
    'Williamsburg': 'Trendy_Hip', 'Bushwick': 'Trendy_Hip', 'Greenpoint': 'Trendy_Hip', 
    'Astoria': 'Trendy_Hip', 'Long Island City': 'Trendy_Hip', 'Ridgewood': 'Trendy_Hip',
    'East Village': 'Trendy_Hip', 'Lower East Side': 'Trendy_Hip',

    # 3. 안정적인 주거/부촌 (Established Residential)
    'Upper East Side': 'Established_Res', 'Upper West Side': 'Established_Res', 
    'Park Slope': 'Established_Res', 'Brooklyn Heights': 'Established_Res', 
    'Clinton Hill': 'Established_Res', 'Fort Greene': 'Established_Res',

    # 4. 급부상/가성비 주거지역 (Up-and-Coming / Commuter)
    'Bedford-Stuyvesant': 'Up_and_Coming', 'Harlem': 'Up_and_Coming', 
    'Crown Heights': 'Up_and_Coming', 'East Harlem': 'Up_and_Coming', 
    'Flatbush': 'Up_and_Coming', 'Washington Heights': 'Up_and_Coming',
    'Prospect-Lefferts Gardens': 'Up_and_Coming', 'Sunnyside': 'Up_and_Coming',

    # 5. 외곽/교외 지역 (Outer Suburban)
    'Flushing': 'Outer_Suburban', 'Jamaica': 'Outer_Suburban', 'East New York': 'Outer_Suburban',
    'Canarsie': 'Outer_Suburban', 'St. Albans': 'Outer_Suburban', 'Jackson Heights': 'Outer_Suburban'
}

# 데이터프레임 적용
# 딕셔너리에 없는 나머지 동네들은 'Others'로 처리
df['geo_characteristic'] = df['neighbourhood_cleansed'].map(geo_mapping).fillna('Others')

In [9]:
# 동네별 평균 가격을 기준으로 동네들 그룹화 하기

# 1. 동네별 평균 가격 계산 (Series 형태)
neighborhood_avg = df.groupby('neighbourhood_cleansed')['price'].mean()

# 2. 평균 가격을 기준으로 4개 등급(Luxury, High, Mid, Budget)으로 나누기
# qcut은 데이터 개수가 균등하게 배분되도록 등급을 나눕니다.
price_labels = ['Budget', 'Mid-Range', 'High-End', 'Luxury']
df['price_group'] = pd.qcut(df['neighbourhood_cleansed'].map(neighborhood_avg), 
                            q=4, 
                            labels=price_labels)


In [10]:
# 화장실 상한선 적용하기

# (1.5개 같은 화장실을 2.0으로 만듭니다)
df['bathrooms'] = np.ceil(df['bathrooms'])

# 상한선(Capping) 적용
df['bathrooms_clipped'] = df['bathrooms'].clip(upper=5.0)

# 결과 확인
print(df['bathrooms_clipped'].value_counts().sort_index())

bathrooms_clipped
0.0       91
1.0    11112
2.0     1948
3.0      269
4.0       80
5.0       31
Name: count, dtype: int64


In [11]:
# 침실 상한선 적용하기
df['bedrooms_clipped'] = df['bedrooms'].clip(upper=6.0)
df['bedrooms_clipped'].value_counts()

bedrooms_clipped
1.0    8419
2.0    2400
0.0    1479
3.0     908
4.0     221
5.0      70
6.0      34
Name: count, dtype: int64

In [12]:
# 침대 상한선 적용하기
df['beds_clipped'] = df['beds'].clip(upper=9.0)
df['beds_clipped'].value_counts()

beds_clipped
1.0    7478
2.0    3381
3.0    1298
4.0     659
0.0     275
5.0     230
6.0     112
7.0      42
8.0      35
9.0      21
Name: count, dtype: int64

In [16]:
# 편의시설 파생 컬럼

import ast
import re


# 편의시설 파싱 함수
def clean_amenities(x):
    if pd.isna(x): return []
    x = x.replace('{', '[').replace('}', ']')
    try:
        return [i.strip().lower() for i in ast.literal_eval(x)]
    except:
        return [i.strip().lower() for i in re.sub(r'[\[\]\"\'\{\}]', '', x).split(',') if i.strip()]

df['amenities_list'] = df['amenities'].apply(clean_amenities)


# 카테고리 정의 (키워드 기반)
categories = {
    'luxury_score': ['elevator', 'gym', 'pool'],
    'service_score': ['washer', 'coffee maker', 'self check-in', 'pet'],
    'longterm_score': ['shampoo', 'dryer', 'iron'],
    'essential_score' : ['air conditioning', 'essentials']
}

# 각 카테고리별 점수 계산 (보유한 항목의 개수)
for cat_name, keywords in categories.items():
    df[cat_name] = df['amenities_list'].apply(lambda x: sum(1 for item in x if any(k in item for k in keywords)))

    # 점수들과 log_price 간의 상관관계 계산
score_cols = list(categories.keys()) + ['amenity_count']
df['amenity_count'] = df['amenities_list'].apply(len)

In [17]:
# 'amenities_list' 컬럼 제거
df.drop(columns=['amenities_list'], inplace=True)

In [15]:
df.columns

Index(['neighbourhood_cleansed', 'neighbourhood_group_cleansed',
       'property_type', 'room_type', 'accommodates', 'bathrooms', 'bedrooms',
       'beds', 'amenities', 'minimum_nights', 'maximum_nights', 'price',
       'geo_characteristic', 'price_group', 'bathrooms_clipped',
       'bedrooms_clipped', 'beds_clipped', 'luxury_score', 'service_score',
       'longterm_score', 'essential_score'],
      dtype='object')

# 분석에 사용할 컬럼 소개

### **원래 있던 컬럼**

---------------------------------------------------------------

**지역 관련 컬럼**
- **neighbourhood_cleansed** : 정제된 동네 이름
- **neighbourhood_group_cleansed** : 정제된 구/군 단위 지역 그룹

---------------------------------------------------------------

**숙소 시설 및 상세 사양**
- **property_type**: 숙소 유형
- **room_type**: 방 유형
- **accommodates**: 수용 인원
- **Luxury_Score**: 욕실 개수
- **bedrooms**: 침실 개수
- **beds**: 침대 개수
- **amenities**: 편의시설 목록

---------------------------------------------------------------

**예약 규칙 및 가용성**
- **minimum_nights**: 최소 숙박 일수
- **maximum_nights**: 최대 숙박 일수
---------------------------------------------------------------

### **새롭게 만든 컬럼**

---------------------------------------------------------------

**지역 관련 컬럼**
- **geo_characteristic** : 지역적 특성에 따라 지역을 분류
- **price_group** : 평균 가격에 따라 지역을 분류

---------------------------------------------------------------

**숙소 시설 및 상세 사양**
- **bathrooms_clipped**: 화장실에 제한 0.5개는 올
- **bedrooms_clipped**: 침실에 제한
- **beds_clipped**: 침대에 제한
- 
---------------------------------------------------------------

**예약 규칙 및 가용성**
- **luxury_score**: 편의 시설 중 'elevator', 'gym', 'pool' 이 포함된 개수
- **service_score**: 편의 시설 중 'washer', 'coffee maker', 'self check-in', 'pet' 이 포함된 개수
- **longterm_score**: 편의 시설 중 'shampoo', 'dryer', 'iron' 이 포함된 개수
- **amenity_count**: 편의시설 개수
---------------------------------------------------------------